<a href="https://colab.research.google.com/github/CUSPADS2022IBX/IBXRidership/blob/main/PLUTO_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PLUTO Cleaning

In [3]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=9ade14f180b945c7d3dc9b1ce3078fc7ccd457b9f9ac5ecbaa08dbe936865eb7
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [4]:
!pip install pyspark
!pip install pandas fiona shapely pyproj rtree
!pip install geopandas


     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 64.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=a550eea4b02fbc2b2e6c75ec2d9471c33eff39450b157af1f79a224acc61928e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 16.7 MB 7.9 MB/s 
     |████████████████████████████████| 6.3 MB 49.2 MB/s 
     |████████████████████████████████| 1.0 MB 34.5 MB/s 
     |████████████████████████████████| 1.0 MB 7.9 MB/s 


In [33]:
import csv
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import IPython
import wget
%matplotlib inline
IPython.display.set_matplotlib_formats('svg')
pd.plotting.register_matplotlib_converters()
sns.set_style("whitegrid")
import zipfile

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
sc = pyspark.SparkContext.getOrCreate()
spark = SparkSession(sc)
spark

#### Reading in PLUTO .csv directly from zip URL

In [6]:
#downloading
!wget 'https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_21v4_csv.zip'
!unzip 'nyc_pluto_21v4_csv.zip'

--2022-04-20 19:13:29--  https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_21v4_csv.zip
Resolving www1.nyc.gov (www1.nyc.gov)... 104.100.66.47, 2600:1406:3c:38e::1500, 2600:1406:3c:383::1500
Connecting to www1.nyc.gov (www1.nyc.gov)|104.100.66.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93698884 (89M) [application/zip]
Saving to: ‘nyc_pluto_21v4_csv.zip’

nyc_pluto_21v4_csv. 100%[===================>]  89.36M  15.9MB/s    in 5.8s    

2022-04-20 19:13:35 (15.4 MB/s) - ‘nyc_pluto_21v4_csv.zip’ saved [93698884/93698884]

Archive:  nyc_pluto_21v4_csv.zip
  inflating: PLUTODD21v4.pdf         
  inflating: PlutoReadme21v4.pdf     
  inflating: pluto_21v4.csv          


In [34]:
#reading as spark dataframe 
pluto = spark.read.csv('/content/pluto_21v4.csv',header=True)\
.select('BCT2020','lotarea', 'bldgarea', 'comarea', 'resarea',
       'officearea', 'retailarea', 'garagearea', 'strgearea', 'factryarea',
       'otherarea','numbldgs','numfloors','unitsres','unitstotal','assesstot')


In [35]:
#list of column names for iterating 
cols = pluto.columns[1:]

#iterating through spark DF columns to cast to float 
for col_name in cols:
    pluto = pluto.withColumn(col_name, F.col(col_name).cast('float'))

#sums of counts as groupby
pluto_tract = pluto.groupBy('BCT2020').sum()

#list of column names for iterating
cols1 = pluto_tract.columns[1:]

for i in range(len(cols1)):
    pluto_tract = pluto_tract.withColumnRenamed(cols1[i],cols[i])

In [36]:
pluto_tract.show(5)


#aggregated by station 

+-------+---------+---------+--------+---------+----------+----------+----------+---------+----------+---------+--------+------------------+--------+----------+------------+
|BCT2020|  lotarea| bldgarea| comarea|  resarea|officearea|retailarea|garagearea|strgearea|factryarea|otherarea|numbldgs|         numfloors|unitsres|unitstotal|   assesstot|
+-------+---------+---------+--------+---------+----------+----------+----------+---------+----------+---------+--------+------------------+--------+----------+------------+
|3056600|1258182.0|1254905.0|196353.0| 978845.0|   82590.0|   69635.0|    5550.0|      0.0|       0.0|  38578.0|   575.0|             762.0|   988.0|    1032.0| 5.2304088E7|
|4072300|3602899.0|1899353.0|125662.0|1756701.0|    6791.0|    8100.0|       0.0|      0.0|       0.0| 110771.0|  1045.0|1574.5899999141693|   913.0|     934.0|  9.459345E7|
|4027200|1194838.0| 644470.0| 99013.0| 538597.0|    8594.0|   49467.0|       0.0|   2550.0|       0.0|  36853.0|   617.0| 954.6699

In [ ]:
#could group by census block (~350,169) different groups 
#could aggregate by census tract (~5,000)
#could aggregate by zip (~1700)
#could aggregate by tax lots or tax blocks 

2323

In [37]:
pluto_df = pluto_tract.toPandas()
pluto_df.head()

,BCT2020,lotarea,bldgarea,comarea,resarea,officearea,retailarea,garagearea,strgearea,factryarea,otherarea,numbldgs,numfloors,unitsres,unitstotal,assesstot
0,3056600,1258182.0,1254905.0,196353.0,978845.0,82590.0,69635.0,5550.0,0.0,0.0,38578.0,575.0,762.00,988.0,1032.0,52304088.0
1,4072300,3602899.0,1899353.0,125662.0,1756701.0,6791.0,8100.0,0.0,0.0,0.0,110771.0,1045.0,1574.59,913.0,934.0,94593450.0
2,4027200,1194838.0,644470.0,99013.0,538597.0,8594.0,49467.0,0.0,2550.0,0.0,36853.0,617.0,954.67,550.0,595.0,21781104.0
3,3080200,1428334.0,2334285.0,300431.0,1947838.0,13892.0,93475.0,1215.0,0.0,0.0,191849.0,464.0,1096.43,1924.0,2041.0,103270263.0
4,2033700,2948047.0,1887595.0,370078.0,1400446.0,128057.0,66977.0,36040.0,1041.0,0.0,136467.0,663.0,1254.76,1256.0,1352.0,66661442.0


In [11]:
import geopandas as gpd

In [12]:
#getting census data 
!wget 'https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyct2020_22a.zip'
!unzip 'nyct2020_22a.zip'



--2022-04-20 19:14:11--  https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyct2020_22a.zip
Resolving www1.nyc.gov (www1.nyc.gov)... 104.100.66.47, 2600:1406:3c:383::1500, 2600:1406:3c:38e::1500
Connecting to www1.nyc.gov (www1.nyc.gov)|104.100.66.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1472942 (1.4M) [application/zip]
Saving to: ‘nyct2020_22a.zip’

nyct2020_22a.zip    100%[===================>]   1.40M  --.-KB/s    in 0.1s    

2022-04-20 19:14:12 (13.3 MB/s) - ‘nyct2020_22a.zip’ saved [1472942/1472942]

Archive:  nyct2020_22a.zip
  inflating: nyct2020_22a/nyct2020.shp  
  inflating: nyct2020_22a/nyct2020.dbf  
  inflating: nyct2020_22a/nyct2020.shx  
  inflating: nyct2020_22a/nyct2020.prj  
  inflating: nyct2020_22a/nyct2020.shp.xml  


In [13]:
#read in shapefile for later merge 
census_geo = gpd.read_file('/content/nyct2020_22a/nyct2020.dbf')

census_geo.shape


(2325, 14)

In [14]:
#read in ACS census data from git 
census_data = pd.read_csv('https://raw.githubusercontent.com/CUSPADS2022IBX/IBXRidership/main/PLUTO%20Data%20Processing/census_tract_data_nyc.csv')

In [38]:
#merge pluto with census data 
pluto_census = pluto_df.merge(census_data.drop_duplicates('BCT2020'),left_on='BCT2020',right_on='BCT2020', how = 'left')
pluto_census.shape

(2323, 51)

In [39]:
#cleaning census columns from obejects to float 
#remove commas
pluto_census = pluto_census.replace(',','', regex=True)

#fill nas with 0s 

In [40]:
#convert to float
cols = pluto_census.columns.drop(['BCT2020','Name', 'CD Type', 'NTA Type'])

pluto_census[cols] = pluto_census[cols].apply(pd.to_numeric, errors='coerce')

In [41]:
#drop weird last column
pluto_census = pluto_census.iloc[:,:-1]



In [42]:
#merge with census geometry
pluto_geo = pluto_census.merge(census_geo,left_on='BCT2020',right_on='BoroCT2020',how='left')
pluto_geo.shape

(2323, 64)

,BCT2020,lotarea,bldgarea,comarea,resarea,officearea,retailarea,garagearea,strgearea,factryarea,...,other_num,other_perc,multirace_num,multirace_perc,total_houses,occupied_homes_num,occupied_homes_perc,vacant_homes_num,vacant_homes_perc,Unnamed: 35
0,3056600,1258182.0,1254905.0,196353.0,978845.0,82590.0,69635.0,5550.0,0.0,0.0,...,20,0.8,91,3.6,"1,049",990,94.4,59,5.6,NaN
1,4072300,3602899.0,1899353.0,125662.0,1756701.0,6791.0,8100.0,0.0,0.0,0.0,...,21,1.0,99,4.6,858,799,93.1,59,6.9,NaN
2,4027200,1194838.0,644470.0,99013.0,538597.0,8594.0,49467.0,0.0,2550.0,0.0,...,138,6.7,156,7.6,615,591,96.1,24,3.9,NaN
3,3080200,1428334.0,2334285.0,300431.0,1947838.0,13892.0,93475.0,1215.0,0.0,0.0,...,59,1.3,320,7.1,"1,999","1,881",94.1,118,5.9,NaN
4,2033700,2948047.0,1887595.0,370078.0,1400446.0,128057.0,66977.0,36040.0,1041.0,0.0,...,70,1.8,207,5.4,"1,288","1,182",91.8,106,8.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318,1000500,7736692.0,2598091.0,2598091.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,NaN,2,40.0,25,0,NaN,25,100.0,NaN
2319,1000100,3306076.0,1145016.0,1145016.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,NaN,0,NaN,0,0,NaN,0,NaN,NaN
2320,3015400,10490000.0,16570.0,16570.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,11.1,3,33.3,1,0,NaN,1,100.0,NaN
2321,1023802,490397.0,661538.0,661538.0,0.0,0.0,0.0,0.0,0.0,0.0,...,223,4.7,211,4.4,"1,983","1,802",90.9,181,9.1,NaN


In [43]:
#creating geodf 
pluto_gdf = gpd.GeoDataFrame(
    pluto_geo, geometry=pluto_geo['geometry'],crs=census_geo.crs)

In [21]:
#create voronoi polyongs from station points 
stations = pd.read_csv('https://raw.githubusercontent.com/CUSP2021ADS/Data/main/NYCSubwayNodes.csv'); stations

,name,id,boro,lat,long,lines
0,Greenpoint Av,G26,Bk,40.731352,-73.954449,G
1,21 St,G24,Q,40.744065,-73.949724,G
2,Court Sq,G22,Q,40.746554,-73.943832,G
3,36 St,G20,Q,40.752039,-73.928781,M R
4,Queens Plaza,G21,Q,40.748973,-73.937243,E M R
...,...,...,...,...,...,...
493,City Hall,R24,M,40.713282,-74.006978,R W
494,Canal St,R23,M,40.719527,-74.001775,R W
495,Prince St,R22,M,40.724329,-73.997702,R W
496,8 St - NYU,R21,M,40.730328,-73.992629,R W


In [ ]:
#download borough boundary from:
#https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm
#OR https://github.com/CUSPADS2022IBX/IBXRidership/tree/main/PLUTO%20Data%20Processing/Borough%20Boundaries


In [22]:
#getting shapefiles from link directly 
!wget 'https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Shapefile'
!unzip 'tqmj-j8zm?method=export&format=Shapefile'


--2022-04-20 19:14:13--  https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Shapefile
Resolving data.cityofnewyork.us (data.cityofnewyork.us)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cityofnewyork.us (data.cityofnewyork.us)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘tqmj-j8zm?method=export&format=Shapefile’

tqmj-j8zm?method=ex     [  <=>               ] 908.98K  2.93MB/s    in 0.3s    

2022-04-20 19:14:15 (2.93 MB/s) - ‘tqmj-j8zm?method=export&format=Shapefile’ saved [930791]

Archive:  tqmj-j8zm?method=export&format=Shapefile
  inflating: geo_export_aab5dd08-abfc-4bcd-a889-7fd4fc79e3ba.dbf  
  inflating: geo_export_aab5dd08-abfc-4bcd-a889-7fd4fc79e3ba.shp  
  inflating: geo_export_aab5dd08-abfc-4bcd-a889-7fd4fc79e3ba.shx  
  inflating: geo_export_aab5dd08-abfc-4bcd-a889-7fd4fc79e3ba.prj  


In [44]:
#convert to gpd, correct crs
#need to copy file path from content directory
boros = gpd.read_file('/content/geo_export_aab5dd08-abfc-4bcd-a889-7fd4fc79e3ba.dbf')

boros


,boro_code,boro_name,shape_area,shape_leng,geometry
0,4.0,Queens,3.040206e+09,900269.280485,"MULTIPOLYGON (((-73.82645 40.59053, -73.82642 ..."
1,3.0,Brooklyn,1.934167e+09,728477.954082,"MULTIPOLYGON (((-73.86327 40.58388, -73.86381 ..."
2,5.0,Staten Island,1.623631e+09,325924.002076,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ..."
3,1.0,Manhattan,6.366208e+08,359993.126318,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ..."
4,2.0,Bronx,1.187194e+09,463868.599917,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ..."


In [24]:
!pip install contextily
!pip install geovoronoi
!pip install shapely

     |████████████████████████████████| 19.3 MB 62.7 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [25]:
#imports and install for voronoi
import numpy as np
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from shapely.ops import cascaded_union
from geovoronoi.plotting import subplot_for_map, plot_voronoi_polys_with_points_in_area
from geovoronoi import voronoi_regions_from_coords, points_to_coords

In [45]:
#convert stations to geopandas, matching crs 
stations_geo = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations.long, stations.lat),crs=boros.crs)

In [46]:
fig, ax = plt.subplots(figsize=(12, 10))
boros.plot(ax=ax)
stations_geo.plot(ax=ax, markersize=3.5, color = 'black')
plt.show()

In [47]:
#keep doing voronoi
boros_shape = cascaded_union(boros.geometry)
coords = points_to_coords(stations_geo.geometry)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  


In [48]:
#get voronois - takes some time 
poly_shapes, pts = voronoi_regions_from_coords(coords, boros_shape)

In [49]:
#voronoi plot 
fig, ax = subplot_for_map(figsize=(12, 10))


plot_voronoi_polys_with_points_in_area(ax, boros_shape, poly_shapes, coords, pts)
plt.show()

/usr/local/lib/python3.7/dist-packages/descartes/patch.py:65: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  for t in polygon])
/usr/local/lib/python3.7/dist-packages/descartes/patch.py:65: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  for t in polygon])


In [50]:
#some multipolygons turning into polygon lists
voronoi_regions=pd.DataFrame.from_dict(poly_shapes,orient='index')
voronoi_regions

#convert to geopandas
voronoi_gpd = gpd.GeoDataFrame(voronoi_regions, geometry=voronoi_regions[0],crs=boros.crs)

/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:1990: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values


In [51]:
#convert voronoi to new york projection, same as pluto_gdf
voronoi_gpd = voronoi_gpd.to_crs(crs=pluto_gdf.crs)

In [52]:
#!pip install rtree
#!pip install pygeos
import rtree
#import pygeos

In [53]:
#creating polygon IDs for later grouping from index values 
voronoi_gpd = voronoi_gpd.reset_index()


In [54]:
#rename column 
voronoi_gpd = voronoi_gpd.rename(columns={"index": "VoroID"})

In [55]:
voronoi_census = gpd.overlay(voronoi_gpd,pluto_gdf, how='intersection')
#voronoi_census = gpd.sjoin(voronoi_gpd,pluto_gdf, how='right')

#maybe - voronoi_census.area from this - and divide by matching census id 
#voronoi_census.plot(alpha=0.5, edgecolor='k', cmap='tab10');

In [56]:
#getting proportion of area of each intersection between vornoy and census tracts 
voronoi_census['area'] = voronoi_census.area/voronoi_census['Shape_Area']

In [ ]:
#multiply calculated area proportion to all numeric columns 

In [57]:
#column subset for numeric values to multiply 
cols = voronoi_census.select_dtypes(np.number).columns.drop(['VoroID','Shape_Leng', 'Shape_Area', 'area','GeoID','GeoType',
       'Borough'])
cols

Index(['lotarea', 'bldgarea', 'comarea', 'resarea', 'officearea', 'retailarea',
       'garagearea', 'strgearea', 'factryarea', 'otherarea', 'numbldgs',
       'numfloors', 'unitsres', 'unitstotal', 'assesstot', 'Total_pop_Num',
       'Total_pop_perc', 'housed_pop_Num', 'housed_pop_perc',
       'group_house_pop_num', 'group_house_pop_perc', 'industrial_pop_num',
       'industrial_pop_perc', 'avg_household', 'under18_num', 'under18_perc',
       'hispanic_num', 'hispanic_perc', 'white_num', 'white_perc', 'black_num',
       'black_perc', 'asian_num', 'asian_perc', 'other_num', 'other_perc',
       'multirace_num', 'multirace_perc', 'total_houses', 'occupied_homes_num',
       'occupied_homes_perc', 'vacant_homes_num', 'vacant_homes_perc'],
      dtype='object')

In [58]:
#multiply area proportion by column subset 
for col in cols:
    voronoi_census[col] = voronoi_census[col]*voronoi_census['area']


In [59]:
#groupby voronoi IDs and aggregate/add up all census data 
temp = voronoi_census.groupby('VoroID')[cols].sum().reset_index()
temp

,VoroID,lotarea,bldgarea,comarea,resarea,officearea,retailarea,garagearea,strgearea,factryarea,...,asian_perc,other_num,other_perc,multirace_num,multirace_perc,total_houses,occupied_homes_num,occupied_homes_perc,vacant_homes_num,vacant_homes_perc
0,0,1.222652e+08,1.677773e+07,2.928741e+06,1.265250e+07,5.194904e+05,3.795373e+05,123068.511583,222712.434657,1.336429e+06,...,65.779154,137.631907,4.017546,511.007396,19.394751,9180.676546,8730.638438,618.984589,450.038107,31.597458
1,1,1.323635e+08,3.124944e+07,6.731926e+06,2.353292e+07,1.415388e+06,1.823002e+06,263290.114930,547583.178879,6.580139e+05,...,161.241603,618.049672,13.192429,1445.728599,30.851652,21409.135407,20254.125056,1165.926912,1155.010351,69.019452
2,2,1.313760e+07,7.207002e+06,2.910237e+06,4.246064e+06,1.278707e+06,4.816201e+05,424345.152816,45808.606916,1.243113e+05,...,22.987709,139.671367,3.857041,421.111185,11.364007,4325.648170,4019.839514,243.499018,305.808656,19.869806
3,3,8.608664e+06,1.665209e+06,2.313913e+05,1.364019e+06,3.554237e+04,8.667865e+04,3198.188584,44541.271005,1.515508e+04,...,1.206228,16.987832,0.330367,50.342213,1.014277,926.992415,848.384149,44.339986,78.608266,4.125088
4,4,7.366622e+07,1.752584e+07,3.741843e+06,1.313043e+07,5.785019e+05,1.179233e+06,85882.148180,54995.635973,3.817032e+05,...,135.770246,307.275587,16.445587,764.742912,22.001906,11792.984390,11099.127724,870.923130,693.856667,65.286285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,487,8.977352e+06,1.784151e+06,5.626756e+05,1.166142e+06,5.361255e+04,1.320115e+05,6816.850449,4686.916749,9.132064e+03,...,1.265197,32.869150,0.728726,67.118436,1.532711,1264.652378,1117.544295,49.280115,147.108083,6.453946
488,488,1.795556e+07,2.946277e+06,6.294043e+05,2.221794e+06,4.409695e+03,3.616173e+04,71028.207760,267.022557,4.929829e+04,...,3.544309,128.117907,3.870628,274.338460,29.722741,2558.036928,2448.925931,263.741126,109.110998,8.920765
489,489,1.108035e+08,2.495126e+07,1.373185e+07,1.070035e+07,1.959111e+05,2.275583e+05,50464.887446,262428.036884,5.228510e+03,...,8.374503,157.453730,12.616776,343.304915,9.673299,9389.578847,7716.019994,583.601757,1673.558852,256.152565
490,490,4.991402e+06,2.702610e+06,9.185778e+05,1.754184e+06,4.804877e+04,6.349218e+04,21821.079527,94761.135728,1.431258e+05,...,5.320599,65.729045,1.752658,192.524835,5.023619,1793.690048,1665.025963,102.902127,128.664085,7.874354


In [60]:
#merge geometry by VoroID to remap 
voro_clean = temp.merge(voronoi_gpd,on='VoroID')

In [67]:
#creating gpd with geometry
voro_clean_gpd = gpd.GeoDataFrame(voro_clean, geometry=voro_clean['geometry'],crs=voronoi_gpd.crs)

In [69]:
voro_clean_gpd.crs

<Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - New York - counties of Bronx; Kings; Nassau; New York; Queens; Richmond; Suffolk.
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [70]:
#convert stations to proper crs 
station_test = stations_geo.to_crs(crs=voro_clean_gpd.crs)

In [73]:
#remerge stations via spatial join - should all interest 1 to 1 with voronoi polygons 
#they seem to 
stations_census_final = voro_clean_gpd.sjoin(station_test, how="left", predicate='intersects')


In [77]:
#write to csv
stations_census_final.to_csv('stations_census_final.csv')

In [78]:
#download bus shapefile and try to do raw count 
#downloading
!wget 'http://web.mta.info/developers/data/nyct/bus/Bus_Shapefiles.zip'
!unzip 'Bus_Shapefiles.zip'

#http://web.mta.info/developers/data/nyct/bus/Bus_Shapefiles.zip

--2022-04-20 19:48:57--  http://web.mta.info/developers/data/nyct/bus/Bus_Shapefiles.zip
Resolving web.mta.info (web.mta.info)... 23.215.102.179, 23.215.102.176
Connecting to web.mta.info (web.mta.info)|23.215.102.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘Bus_Shapefiles.zip’

Bus_Shapefiles.zip      [               <=>  ]   2.89M   965KB/s    in 3.1s    

2022-04-20 19:49:01 (965 KB/s) - ‘Bus_Shapefiles.zip’ saved [3034101]

Archive:  Bus_Shapefiles.zip
  inflating: NYCT Bus Routes.shx     
  inflating: BusStopsAsOfMarch2.dbf  
  inflating: BusStopsAsOfMarch2.prj  
  inflating: BusStopsAsOfMarch2.shp  
  inflating: BusStopsAsOfMarch2.shx  
  inflating: Express_Bus_Routes.cpg  
  inflating: Express_Bus_Routes.dbf  
  inflating: Express_Bus_Routes.prj  
  inflating: Express_Bus_Routes.sbn  
  inflating: Express_Bus_Routes.sbx  
  inflating: Express_Bus_Routes.shp  
  inflating: Express_Bus_Routes.shp.xml  
  inf

In [85]:
bus_stops = gpd.read_file('/content/BusStopsAsOfMarch2.dbf')
bus_stops.head(5)

,box_id,name,zip_name,geometry
0,100646,BOSTON RD/E 180 ST,2016Jan_Prod_r03_b6,POINT (-73.87833 40.84256)
1,103374,BRUSH AV/BRUCKNER BL,2016Jan_Prod_r03_b6,POINT (-73.84096 40.82810)
2,103379,CROSS BX SVC RD/NEWBOLD AV,2016Jan_Prod_r03_b6,POINT (-73.85699 40.83250)
3,103380,CROSS BX SVC RD/TAYLOR AV,2016Jan_Prod_r03_b6,POINT (-73.86643 40.83539)
4,103381,CROSS BX SVC RD/COMMONWEALTH AV,2016Jan_Prod_r03_b6,POINT (-73.87041 40.83643)


In [92]:
#convert crs 
bus_stops = bus_stops.to_crs(crs=stations_census_final.crs)

In [100]:
#drop index right column for proper merge
test = stations_census_final.drop('index_right',axis=1)

In [102]:
#merge in number of bus stops intersecting in each voronoi polygon/census area
#first general interesection 
station_bus_join = test.sjoin(bus_stops,how='left',predicate='intersects')


In [121]:
station_bus_join.head(3)

,VoroID,lotarea,bldgarea,comarea,resarea,officearea,retailarea,garagearea,strgearea,factryarea,...,name_left,id,boro,lat,long,lines,index_right,box_id,name_right,zip_name
0,0,1.222652e+08,1.677773e+07,2.928741e+06,1.265250e+07,519490.372629,379537.304593,123068.511583,222712.434657,1.336429e+06,...,Annadale,S17,SI,40.54046,-74.178217,SIR,13507.0,905114,WEST SHORE PLAZA/CHELSEA ST,2016Jan_Prod_r03_b6
0,0,1.222652e+08,1.677773e+07,2.928741e+06,1.265250e+07,519490.372629,379537.304593,123068.511583,222712.434657,1.336429e+06,...,Annadale,S17,SI,40.54046,-74.178217,SIR,12928.0,203386,WEST SHORE PLAZA/CHELSEA ST,2016Jan_Prod_r03_b6
0,0,1.222652e+08,1.677773e+07,2.928741e+06,1.265250e+07,519490.372629,379537.304593,123068.511583,222712.434657,1.336429e+06,...,Annadale,S17,SI,40.54046,-74.178217,SIR,12490.0,202055,VICTORY BL/GLEN ST,2016Jan_Prod_r03_b6


In [122]:
len(station_bus_join['index_right'].unique())

15307

In [128]:
#pivotting to get counts before merge - counting up box id for raw bus counts
tmp2 = pd.pivot_table(station_bus_join,index='VoroID',columns='box_id',aggfunc={'box_id':len}).fillna(0)

In [130]:
tmp2.columns = tmp2.columns.droplevel()


box_id,100014,100017,100018,100019,100020,100021,100022,100025,100026,100027,...,982069,982072,983012,984006,984009,984010,984012,984013,984014,999999
VoroID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [131]:
#getting total count
tmp2['Bus_Stops'] = tmp2.sum(axis=1)

In [134]:
#larger polygons have more buses - could be heavilt skewed would be wary of this 

tmp2 = tmp2[['Bus_Stops']]

In [136]:
#reset index for merge
tmp2 = tmp2.reset_index()
tmp2

box_id,VoroID,Bus_Stops
0,0,79.0
1,1,223.0
2,2,60.0
3,3,12.0
4,4,147.0
...,...,...
481,487,6.0
482,488,8.0
483,489,76.0
484,490,12.0


In [142]:
station_bus_final = test.merge(tmp2,on='VoroID',how='left')


In [143]:
station_bus_final

/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/inference.py:383: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  iter(obj)  # Can iterate over it.
/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/inference.py:384: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  len(obj)  # Has a length associated with it.
/usr/local/lib/python3.7/dist-packages/pandas/io/formats/printing.py:118: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  s = iter(seq)
/usr/local/lib/python3.7/dist-packages/pandas/io/formats/printing.p

,VoroID,lotarea,bldgarea,comarea,resarea,officearea,retailarea,garagearea,strgearea,factryarea,...,vacant_homes_perc,0,geometry,name,id,boro,lat,long,lines,Bus_Stops
0,0,1.222652e+08,1.677773e+07,2.928741e+06,1.265250e+07,5.194904e+05,3.795373e+05,123068.511583,222712.434657,1.336429e+06,...,31.597458,"POLYGON ((-74.19585750848185 40.6231536160957,...","POLYGON ((929879.914 166368.828, 938061.176 13...",Annadale,S17,SI,40.540460,-74.178217,SIR,79.0
1,1,1.323635e+08,3.124944e+07,6.731926e+06,2.353292e+07,1.415388e+06,1.823002e+06,263290.114930,547583.178879,6.580139e+05,...,69.019452,POLYGON ((-74.10054804434564 40.55855711784107...,"POLYGON ((956310.811 142789.941, 956240.802 14...",New Dorp,S22,SI,40.573480,-74.117210,SIR,223.0
2,2,1.313760e+07,7.207002e+06,2.910237e+06,4.246064e+06,1.278707e+06,4.816201e+05,424345.152816,45808.606916,1.243113e+05,...,19.869806,POLYGON ((-74.10449899062377 40.64565367852967...,"POLYGON ((955250.820 174522.721, 955279.370 17...",St George,S31,SI,40.643748,-74.073643,SIR,60.0
3,3,8.608664e+06,1.665209e+06,2.313913e+05,1.364019e+06,3.554237e+04,8.667865e+04,3198.188584,44541.271005,1.515508e+04,...,4.125088,POLYGON ((-74.24924033389159 40.51507035520215...,"POLYGON ((914948.826 127029.244, 914955.806 12...",Atlantic,S10,SI,40.515401,-74.245689,SIR,12.0
4,4,7.366622e+07,1.752584e+07,3.741843e+06,1.313043e+07,5.785019e+05,1.179233e+06,85882.148180,54995.635973,3.817032e+05,...,65.286285,(POLYGON ((-74.15878823195226 40.6382564520573...,"MULTIPOLYGON (((940180.315 171850.356, 940219....",Grant City,S23,SI,40.578965,-74.109704,SIR,147.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,487,8.977352e+06,1.784151e+06,5.626756e+05,1.166142e+06,5.361255e+04,1.320115e+05,6816.850449,4686.916749,9.132064e+03,...,6.453946,POLYGON ((-73.82302486483574 40.58080877226646...,"POLYGON ((1033409.510 150930.390, 1033360.258 ...",Beach 105 St,H14,Q,40.583209,-73.827559,A S,6.0
492,488,1.795556e+07,2.946277e+06,6.294043e+05,2.221794e+06,4.409695e+03,3.616173e+04,71028.207760,267.022557,4.929829e+04,...,8.920765,POLYGON ((-73.78287847079686 40.60609019366927...,"POLYGON ((1044538.380 160166.111, 1044548.024 ...",Beach 44 St,H08,Q,40.592943,-73.776013,A,8.0
493,489,1.108035e+08,2.495126e+07,1.373185e+07,1.070035e+07,1.959111e+05,2.275583e+05,50464.887446,262428.036884,5.228510e+03,...,256.152565,POLYGON ((-73.83090077277144 40.57849590721296...,"POLYGON ((1031223.399 150083.436, 1031177.789 ...",Rockaway Park - Beach 116 St,H15,Q,40.580903,-73.835592,A S,76.0
494,490,4.991402e+06,2.702610e+06,9.185778e+05,1.754184e+06,4.804877e+04,6.349218e+04,21821.079527,94761.135728,1.431258e+05,...,7.874354,POLYGON ((-73.81558190275469 40.58282113434326...,"POLYGON ((1035475.443 151667.807, 1035460.846 ...",Beach 98 St,H13,Q,40.585307,-73.820558,A S,12.0


In [144]:
#output bus stops as csv 
station_bus_final.to_csv('station_bus_final.csv')

/usr/local/lib/python3.7/dist-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


In [ ]:
#match voronoi to each station ID --> re map 
#look into maintaing stations id from merge above 

In [ ]:
#need to convert census tracts to same crs!!

